In [ ]:
import xarray as xr
import glob
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
# location of 10-min timeseries of surge
sys.path.append("..")
from path_dict import path_dict
dir_postproc = path_dict['postproc']
dir_ts = os.path.join(dir_postproc,'timeseries-GTSM-ERA5-10min','surge')

In [ ]:
# output directory
outdir = os.path.join(dir_postproc,'SeaLevelMonitor')

In [ ]:
# select year to process
yrs = range(1955,2022)

In [ ]:
for yr in yrs:
    print(f'Processing year {yr}...')

    ts_files = glob.glob(os.path.join(dir_ts,f'reanalysis_surge_10min_{yr}_*_v1.nc'))

    if len(ts_files) == 12:
        # open all data for that year in one dataset
        ds = xr.open_mfdataset(ts_files)
        
        # resample to monthly means
        ds_monthly_mean = ds.resample(time='M').mean()
        ds_monthly_mean['station_x_coordinate'] = ds_monthly_mean['station_x_coordinate'].isel(time=0,drop=True)
        ds_monthly_mean['station_y_coordinate'] = ds_monthly_mean['station_y_coordinate'].isel(time=0,drop=True) 
        
        # drop the time dimension on coordinates that got added when opening the dataset
        ds_monthly_mean.load()
        
        # save resulting file
        ofile = os.path.join(outdir,'era5_reanalysis_surge_%s_v1_monthly_mean.nc' % yr)
        ds_monthly_mean.to_netcdf(ofile)
        del ds, ds_monthly_mean
        print('==== Done!')
    else:
        print('==== Not all months are available in %s'% yr)


In [ ]:
# checkplot of timeseries and monthly means for one location

#loc = [4.556043, 52.461978] # IJmuiden station coordinates [lon,lat]

# find station number
#abslon = np.abs(ds.station_x_coordinate-loc[0])
#abslat = np.abs(ds.station_y_coordinate-loc[1])
#station_id = ds['stations'].isel(stations=np.argmin(abslon.values**2 + abslat.values**2)).values.tolist()

In [ ]:
#plt.plot(ds['time'],ds['surge'].sel(stations=station_id))
#for mm in range(1,12):
#    plt.plot([ds_monthly_mean['time'][mm-1].values,ds_monthly_mean['time'][mm].values],[ds_monthly_mean['surge'].sel(stations=station_id)[mm],ds_monthly_mean['surge'].sel(stations=station_id)[mm]],color='red')